In [16]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Load Data

In [17]:
DATA_FOLDER="../../00_Data"

train_raw_df = pd.read_excel(os.path.join(DATA_FOLDER, "telco_train.xlsx"))
train_raw_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


# Data Subset

In [22]:
dept_job_role_df = train_raw_df.filter(['EmployeeNumber', 'Department', 'JobRole', 'PerformanceRating', 'Attrition'])
dept_job_role_df.head()

,EmployeeNumber,Department,JobRole,PerformanceRating,Attrition
0,1,Sales,Sales Executive,3,Yes
1,2,Research & Development,Research Scientist,4,No
2,4,Research & Development,Laboratory Technician,3,Yes
3,5,Research & Development,Research Scientist,3,No
4,7,Research & Development,Laboratory Technician,3,No


# Business Science Problem Framework

#### 1A. View Business as Machine
* Business Unities: Department and Job Role
* Define Objectives: Retain High Performers
* Assess Outcomes: TBD

In [30]:
(
    dept_job_role_df
    .groupby('Attrition')
    .agg(n=('Attrition', 'count'))
    .reset_index()
    .assign(pct=lambda df: df.n / df.n.sum())
)

,Attrition,n,pct
0,No,1049,0.8392
1,Yes,201,0.1608


#### 1B. Understand the Drivers

* Investigate Objectives: 16% Attrition
* Synthesize Outcomes: High Counts and High percentages
* Hypothesize Drivers: Job Role and Departments

Department

In [40]:
(
    dept_job_role_df
    .groupby(['Department', 'Attrition'])
    .agg(n=('Attrition', 'count'))
    .reset_index()
    .assign(pct=lambda df: df.n / df.groupby('Department')['n'].transform('sum'))
)

,Department,Attrition,n,pct
0,Human Resources,No,37,0.755102
1,Human Resources,Yes,12,0.244898
2,Research & Development,No,721,0.866587
3,Research & Development,Yes,111,0.133413
4,Sales,No,291,0.788618
5,Sales,Yes,78,0.211382


JobRole

In [45]:
(
    dept_job_role_df
    .groupby(['Department', 'JobRole', 'Attrition'])
    .agg(n=('Attrition', 'count'))
    .reset_index()
    .assign(pct=lambda df: df.n / df.groupby(['Department', 'JobRole'])['n'].transform('sum'))
    .query('Attrition == "Yes"')
    .sort_values(by='pct', ascending=False)
)

,Department,JobRole,Attrition,n,pct
20,Sales,Sales Representative,Yes,26,0.400000
1,Human Resources,Human Resources,Yes,12,0.307692
6,Research & Development,Laboratory Technician,Yes,49,0.218750
18,Sales,Sales Executive,Yes,50,0.183150
14,Research & Development,Research Scientist,Yes,43,0.166023
4,Research & Development,Healthcare Representative,Yes,8,0.076190
16,Sales,Manager,Yes,2,0.064516
10,Research & Development,Manufacturing Director,Yes,7,0.056911
8,Research & Development,Manager,Yes,2,0.041667
12,Research & Development,Research Director,Yes,2,0.027397


#### 1C. Measure The Drivers ----

* Collect Information on Employee Attrition: On going task
* Develops KPI's: Industry KPIs: 8.8%

In [47]:
(
    dept_job_role_df
    .groupby(['Department', 'JobRole', 'Attrition'])
    .agg(n=('Attrition', 'count'))
    .reset_index()
    .assign(pct=lambda df: df.n / df.groupby(['Department', 'JobRole'])['n'].transform('sum'))
    .query('Attrition == "Yes"')
    .sort_values(by='pct', ascending=False)
    .assign(above_industry_avg=lambda df: np.where(df.pct > 0.088, "Yes", "No"))
)

,Department,JobRole,Attrition,n,pct,above_industry_avg
20,Sales,Sales Representative,Yes,26,0.400000,Yes
1,Human Resources,Human Resources,Yes,12,0.307692,Yes
6,Research & Development,Laboratory Technician,Yes,49,0.218750,Yes
18,Sales,Sales Executive,Yes,50,0.183150,Yes
14,Research & Development,Research Scientist,Yes,43,0.166023,Yes
4,Research & Development,Healthcare Representative,Yes,8,0.076190,No
16,Sales,Manager,Yes,2,0.064516,No
10,Research & Development,Manufacturing Director,Yes,7,0.056911,No
8,Research & Development,Manager,Yes,2,0.041667,No
12,Research & Development,Research Director,Yes,2,0.027397,No


#### 1D. Uncover Problems & Opportunities